In [1]:
from __future__ import print_function
import os
import os.path
from os import mkdir
import numpy as np
import random
import pickle
import time

import shutil


import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.nn.utils.weight_norm import WeightNorm

In [2]:
def check_dir(path):
    '''
    Create directory if it does not exist.
        path:           Path of directory.
    '''
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def log(log_file_path, string):
    '''
    Write one line of log into screen and file.
        log_file_path: Path of log file.
        string:        String to write in log file.
    '''
    with open(log_file_path, 'a+') as f:
        f.write(string + '\n')
        f.flush()
    print(string)

In [4]:
class Timer():
    def __init__(self):
        self.o = time.time()

    def measure(self, p=1):
        x = (time.time() - self.o) / float(p)
        x = int(x)
        if x >= 3600:
            return '{:.1f}h'.format(x / 3600)
        if x >= 60:
            return '{}m'.format(round(x / 60))
        return '{}s'.format(x)

In [5]:
def top1accuracy(output, target):
    _, pred = output.max(dim=1)
    pred = pred.view(-1)
    target = target.view(-1)
    accuracy = 100 * pred.eq(target).float().mean()
    return accuracy

In [6]:
def one_hot(indices, depth):

    encoded_indicies = torch.zeros(indices.size() + torch.Size([depth])).cuda()
    index = indices.view(indices.size() + torch.Size([1]))
    encoded_indicies = encoded_indicies.scatter_(1, index, 1)

    return encoded_indicies

In [7]:
def top1accuracy_all(output, target,nKbase):

    preds_data = output.data.cpu()
    labels_test_data = target.data.cpu()
    nKbase = nKbase.data.cpu()

    base_ids = torch.nonzero(labels_test_data < nKbase).view(-1)
    novel_ids = torch.nonzero(labels_test_data >= nKbase).view(-1)

    preds_base = preds_data[base_ids, :]
    preds_novel = preds_data[novel_ids, :]

    AccuracyBoth = top1accuracy(preds_data, labels_test_data)
    AccuracyBase = top1accuracy(preds_base[:, :nKbase], labels_test_data[base_ids])
    AccuracyNovel = top1accuracy(preds_novel[:, nKbase:], (labels_test_data[novel_ids] - nKbase))

    return AccuracyBoth, AccuracyBase, AccuracyNovel

In [8]:
def ensure_path(path):
    if os.path.exists(path):
        if input('{} exists, remove? ([y]/n)'.format(path)) != 'n':
            shutil.rmtree(path)
            os.mkdir(path)
    else:
        os.mkdir(path)

In [9]:
def set_gpu(gpu):
    os.environ['CUDA_VISIBLE_DEVICES'] = gpu
    print('using gpu {}'.format(gpu))


In [10]:
def pick_vectors(dic, wnids, is_tensor=False):
    o = next(iter(dic.values()))
    dim = len(o)
    ret = []
    for wnid in wnids:
        v = dic.get(wnid)
        if v is None:
            if not is_tensor:
                v = [0] * dim
            else:
                v = torch.zeros(dim)
        ret.append(v)
    if not is_tensor:
        return torch.FloatTensor(ret)
    else:
        return torch.stack(ret)

In [11]:
def l2_loss(a, b):
    return ((a - b)**2).sum() / (len(a) * 2)


In [12]:
def cosine_similarity(a,b,mask):
    return torch.mm(a[mask],b[:,mask])


In [13]:
def normt_spm(mx, method='in'):
    if method == 'in':
        mx = mx.transpose()
        rowsum = np.array(mx.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv)
        mx = r_mat_inv.dot(mx)
        return mx

    if method == 'sym':
        rowsum = np.array(mx.sum(1))
        r_inv = np.power(rowsum, -0.5).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv)
        mx = mx.dot(r_mat_inv).transpose().dot(r_mat_inv)
        return mx

In [14]:
def spm_to_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack(
            (sparse_mx.row, sparse_mx.col))).long()
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

# Data Loader

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [17]:
!pwd

/content/drive/My Drive


In [18]:
_MINI_IMAGENET_DATASET_DIR = "/content/drive/My Drive/miniImagenet"

In [19]:
!pip install torchnet

In [20]:
import torchnet as tnt
from PIL import Image
import math
import argparse
from tqdm import tqdm
from torch.autograd import Variable

In [21]:
def load_data(file):
    try:
        with open(file, 'rb') as fo:
            data = pickle.load(fo)
        return data
    except:
        with open(file, 'rb') as f:
            u = pickle._Unpickler(f)
            u.encoding = 'latin1'
            data = u.load()
        
        return data

In [22]:
def buildLabelIndex(labels):
    label2inds = {}
    for idx, label in enumerate(labels):
        if label not in label2inds:
            label2inds[label] = []
        label2inds[label].append(idx)

    return label2inds

In [23]:
class MiniImageNet(data.Dataset):

    def __init__(self, phase='train', do_not_use_random_transf=False):

        self.base_folder = 'miniImagenet'
        assert (phase == 'train' or phase == 'val' or phase == 'test')
        self.phase = phase
        self.name = 'MiniImageNet_' + phase

        print('Loading mini ImageNet dataset - phase {0}'.format(phase))

        file_train_categories_train_phase = os.path.join(
            _MINI_IMAGENET_DATASET_DIR,
            'miniImageNet_category_split_train_phase_train.pickle')

        file_train_categories_val_phase = os.path.join(
            _MINI_IMAGENET_DATASET_DIR,
            'miniImageNet_category_split_train_phase_val.pickle')

        file_train_categories_test_phase = os.path.join(
            _MINI_IMAGENET_DATASET_DIR,
            'miniImageNet_category_split_train_phase_test.pickle')

        file_val_categories_val_phase = os.path.join(
            _MINI_IMAGENET_DATASET_DIR,
            'miniImageNet_category_split_val.pickle')

        file_test_categories_test_phase = os.path.join(
            _MINI_IMAGENET_DATASET_DIR,
            'miniImageNet_category_split_test.pickle')

        if self.phase == 'train':
            # During training phase we only load the training phase images of the training categories (aka base categories).
            data_train = load_data(file_train_categories_train_phase)
            self.data = data_train['data']
            self.labels = data_train['labels']

            self.label2ind = buildLabelIndex(self.labels)
            self.labelIds = sorted(self.label2ind.keys())

            self.num_cats = len(self.labelIds)

            self.labelIds_base = self.labelIds

            self.num_cats_base = len(self.labelIds_base)

        elif self.phase == 'val' or self.phase == 'test':
            if self.phase == 'test':
                # load data that will be used for evaluating the recognition accuracy of the base categories.
                data_base = load_data(file_train_categories_test_phase)
                # load data that will be use for evaluating the few-shot recogniton accuracy on the novel categories.
                data_novel = load_data(file_test_categories_test_phase)
            elif self.phase == 'val':
                # load data that will be used for evaluating the recognition accuracy of the base categories.
                data_base = load_data(file_train_categories_val_phase)
                # load data that will be use for evaluating the few-shot recogniton accuracy on the novel categories.
                data_novel = load_data(file_val_categories_val_phase)

            self.data = np.concatenate(
                [data_base['data'], data_novel['data']], axis=0)
            self.labels = data_base['labels'] + data_novel['labels']


            self.label2ind = buildLabelIndex(self.labels)
            self.labelIds = sorted(self.label2ind.keys())
            self.num_cats = len(self.labelIds)

            self.labelIds_base = buildLabelIndex(data_base['labels']).keys()
            self.labelIds_novel = buildLabelIndex(data_novel['labels']).keys()


            self.num_cats_base = len(self.labelIds_base)
            self.num_cats_novel = len(self.labelIds_novel)

            intersection = set(self.labelIds_base) & set(self.labelIds_novel)


            assert (len(intersection) == 0)
        else:
            raise ValueError('Not valid phase {0}'.format(self.phase))

        mean_pix = [x / 255.0 for x in [120.39586422, 115.59361427, 104.54012653]]
        std_pix = [x / 255.0 for x in [70.68188272, 68.27635443, 72.54505529]]
        normalize = transforms.Normalize(mean=mean_pix, std=std_pix)

        if (self.phase == 'test' or self.phase == 'val') or (do_not_use_random_transf == True):
            self.transform = transforms.Compose([
                lambda x: np.asarray(x),
                transforms.ToTensor(),
                normalize
            ])
        else:

            self.transform = transforms.Compose([
                transforms.RandomCrop(84, padding=8),
                # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                transforms.RandomHorizontalFlip(),
                lambda x: np.asarray(x),
                transforms.ToTensor(),
                normalize
            ])

    def __getitem__(self, index):
        img, label = self.data[index], self.labels[index]
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data)



In [24]:
class FewShotDataloader():
    def __init__(self,
                 dataset,
                 nKnovel=5, # number of novel categories.
                 nKbase=-1, # number of base categories.
                 nExemplars=1, # number of training examples per novel category.
                 nTestNovel=15*5, # number of test examples for all the novel categories.
                 nTestBase=15*5, # number of test examples for all the base categories.
                 batch_size=1, # number of training episodes per batch.
                 num_workers=4,
                 epoch_size=2000, # number of batches per epoch.
                 ):

        self.dataset = dataset

        self.phase = self.dataset.phase

        max_possible_nKnovel = (self.dataset.num_cats_base if self.phase=='train'
                                else self.dataset.num_cats_novel)
        assert(nKnovel >= 0 and nKnovel < max_possible_nKnovel)

        self.nKnovel = nKnovel

        max_possible_nKbase = self.dataset.num_cats_base
        nKbase = nKbase if nKbase >= 0 else max_possible_nKbase
        if self.phase=='train' and nKbase > 0:
            nKbase -= self.nKnovel
            max_possible_nKbase -= self.nKnovel

        assert(nKbase >= 0 and nKbase <= max_possible_nKbase)
        self.nKbase = nKbase

        self.nExemplars = nExemplars
        self.nTestNovel = nTestNovel
        self.nTestBase = nTestBase
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        self.num_workers = num_workers
        self.is_eval_mode = (self.phase=='test') or (self.phase=='val')

    def sampleImageIdsFrom(self, cat_id, sample_size=1):
        """
        Samples `sample_size` number of unique image ids picked from the
        category `cat_id` (i.e., self.dataset.label2ind[cat_id]).
        Args:
            cat_id: a scalar with the id of the category from which images will
                be sampled.
            sample_size: number of images that will be sampled.
        Returns:
            image_ids: a list of length `sample_size` with unique image ids.
        """
        assert(cat_id in self.dataset.label2ind)
        assert(len(self.dataset.label2ind[cat_id]) >= sample_size)
        # Note: random.sample samples elements without replacement.
        return random.sample(self.dataset.label2ind[cat_id], sample_size)

    def sampleCategories(self, cat_set, sample_size=1):
        """
        Samples `sample_size` number of unique categories picked from the
        `cat_set` set of categories. `cat_set` can be either 'base' or 'novel'.
        Args:
            cat_set: string that specifies the set of categories from which
                categories will be sampled.
            sample_size: number of categories that will be sampled.
        Returns:
            cat_ids: a list of length `sample_size` with unique category ids.
        """
        if cat_set=='base':
            labelIds = self.dataset.labelIds_base
        elif cat_set=='novel':
            labelIds = self.dataset.labelIds_novel
        else:
            raise ValueError('Not recognized category set {}'.format(cat_set))

        assert(len(labelIds) >= sample_size)
        # return sample_size unique categories chosen from labelIds set of
        # categories (that can be either self.labelIds_base or self.labelIds_novel)
        # Note: random.sample samples elements without replacement.

        return random.sample(labelIds, sample_size)

    def sample_base_and_novel_categories(self, nKbase, nKnovel):
        """
        Samples `nKbase` number of base categories and `nKnovel` number of novel
        categories.
        Args:
            nKbase: number of base categories
            nKnovel: number of novel categories
        Returns:
            Kbase: a list of length 'nKbase' with the ids of the sampled base
                categories.
            Knovel: a list of lenght 'nKnovel' with the ids of the sampled novel
                categories.
        """
        if self.is_eval_mode:
            assert(nKnovel <= self.dataset.num_cats_novel)
            # sample from the set of base categories 'nKbase' number of base
            # categories.
            Kbase = sorted(self.sampleCategories('base', nKbase))
            # sample from the set of novel categories 'nKnovel' number of novel
            # categories.
            Knovel = sorted(self.sampleCategories('novel', nKnovel))
        else:
            # sample from the set of base categories 'nKnovel' + 'nKbase' number
            # of categories.
            cats_ids = self.sampleCategories('base', nKnovel+nKbase)
            assert(len(cats_ids) == (nKnovel+nKbase))
            # Randomly pick 'nKnovel' number of fake novel categories and keep
            # the rest as base categories.
            random.shuffle(cats_ids)
            Knovel = sorted(cats_ids[:nKnovel])
            Kbase = sorted(cats_ids[nKnovel:])

        return Kbase, Knovel

    def sample_test_examples_for_base_categories(self, Kbase, nTestBase):
        """
        Sample `nTestBase` number of images from the `Kbase` categories.
        Args:
            Kbase: a list of length `nKbase` with the ids of the categories from
                where the images will be sampled.
            nTestBase: the total number of images that will be sampled.
        Returns:
            Tbase: a list of length `nTestBase` with 2-element tuples. The 1st
                element of each tuple is the image id that was sampled and the
                2nd elemend is its category label (which is in the range
                [0, len(Kbase)-1]).
        """
        Tbase = []
        if len(Kbase) > 0:
            # Sample for each base category a number images such that the total
            # number sampled images of all categories to be equal to `nTestBase`.
            KbaseIndices = np.random.choice(
                np.arange(len(Kbase)), size=nTestBase, replace=True)
            KbaseIndices, NumImagesPerCategory = np.unique(
                KbaseIndices, return_counts=True)

            for Kbase_idx, NumImages in zip(KbaseIndices, NumImagesPerCategory):
                imd_ids = self.sampleImageIdsFrom(
                    Kbase[Kbase_idx], sample_size=NumImages)
                Tbase += [(img_id, Kbase_idx) for img_id in imd_ids]

        assert(len(Tbase) == nTestBase)

        return Tbase

    def sample_train_and_test_examples_for_novel_categories(
            self, Knovel, nTestNovel, nExemplars, nKbase):
        """Samples train and test examples of the novel categories.
        Args:
    	    Knovel: a list with the ids of the novel categories.
            nTestNovel: the total number of test images that will be sampled
                from all the novel categories.
            nExemplars: the number of training examples per novel category that
                will be sampled.
            nKbase: the number of base categories. It is used as offset of the
                category index of each sampled image.
        Returns:
            Tnovel: a list of length `nTestNovel` with 2-element tuples. The
                1st element of each tuple is the image id that was sampled and
                the 2nd element is its category label (which is in the range
                [nKbase, nKbase + len(Knovel) - 1]).
            Exemplars: a list of length len(Knovel) * nExemplars of 2-element
                tuples. The 1st element of each tuple is the image id that was
                sampled and the 2nd element is its category label (which is in
                the ragne [nKbase, nKbase + len(Knovel) - 1]).
        """

        if len(Knovel) == 0:
            return [], []

        nKnovel = len(Knovel)
        Tnovel = []
        Exemplars = []
        assert((nTestNovel % nKnovel) == 0)
        nEvalExamplesPerClass = int(nTestNovel / nKnovel)

        for Knovel_idx in range(len(Knovel)):
            imd_ids = self.sampleImageIdsFrom(
                Knovel[Knovel_idx],
                sample_size=(nEvalExamplesPerClass + nExemplars))

            imds_tnovel = imd_ids[:nEvalExamplesPerClass]
            imds_ememplars = imd_ids[nEvalExamplesPerClass:]

            Tnovel += [(img_id, nKbase+Knovel_idx) for img_id in imds_tnovel]
            Exemplars += [(img_id, nKbase+Knovel_idx) for img_id in imds_ememplars]
        assert(len(Tnovel) == nTestNovel)
        assert(len(Exemplars) == len(Knovel) * nExemplars)
        random.shuffle(Exemplars)

        return Tnovel, Exemplars

    def sample_episode(self):
        """Samples a training episode."""
        nKnovel = self.nKnovel
        nKbase = self.nKbase
        nTestNovel = self.nTestNovel
        nTestBase = self.nTestBase
        nExemplars = self.nExemplars

        Kbase, Knovel = self.sample_base_and_novel_categories(nKbase, nKnovel)
        Tbase = self.sample_test_examples_for_base_categories(Kbase, nTestBase)
        Tnovel, Exemplars = self.sample_train_and_test_examples_for_novel_categories(
            Knovel, nTestNovel, nExemplars, nKbase)

        # concatenate the base and novel category examples.
        Test = Tbase + Tnovel
        random.shuffle(Test)
        Kall = Kbase + Knovel

        return Exemplars, Test, Kall, nKbase

    def createExamplesTensorData(self, examples):
        """
        Creates the examples image and label tensor data.
        Args:
            examples: a list of 2-element tuples, each representing a
                train or test example. The 1st element of each tuple
                is the image id of the example and 2nd element is the
                category label of the example, which is in the range
                [0, nK - 1], where nK is the total number of categories
                (both novel and base).
        Returns:
            images: a tensor of shape [nExamples, Height, Width, 3] with the
                example images, where nExamples is the number of examples
                (i.e., nExamples = len(examples)).
            labels: a tensor of shape [nExamples] with the category label
                of each example.
        """
        images = torch.stack(
            [self.dataset[img_idx][0] for img_idx, _ in examples], dim=0)
        labels = torch.LongTensor([label for _, label in examples])
        return images, labels

    def get_iterator(self, epoch=0):
        rand_seed = epoch
        random.seed(rand_seed)
        np.random.seed(rand_seed)
        def load_function(iter_idx):
            Exemplars, Test, Kall, nKbase = self.sample_episode()
            Xt, Yt = self.createExamplesTensorData(Test)
            Kall = torch.LongTensor(Kall)
            if len(Exemplars) > 0:
                Xe, Ye = self.createExamplesTensorData(Exemplars)
                return Xe, Ye, Xt, Yt, Kall, nKbase
            else:
                return Xt, Yt, Kall, nKbase

        tnt_dataset = tnt.dataset.ListDataset(
            elem_list=range(self.epoch_size), load=load_function)
        data_loader = tnt_dataset.parallel(
            batch_size=self.batch_size,
            num_workers=(0 if self.is_eval_mode else self.num_workers),
            shuffle=(False if self.is_eval_mode else True))

        return data_loader

    def __call__(self, epoch=0):
        return self.get_iterator(epoch)

    def __len__(self):
        return int(self.epoch_size / self.batch_size)

# Convolution model

In [25]:
# Convolution module
class ConvBlock(nn.Module):
    # Initialize the instance object
    def __init__(self, in_planes, out_planes, userelu=True):
        # Initialize the properties inherited from the parent class
        super(ConvBlock, self).__init__()
        # Time series container, quickly build neural network
        self.layers = nn.Sequential()
        # Convolutional layer, the size of the convolution kernel is 3*3, the step size is 1, the padding is 1
        self.layers.add_module('Conv', nn.Conv2d(in_planes, out_planes,
            kernel_size=3, stride=1, padding=1, bias=False))

        # batchnorm Floor
        self.layers.add_module('BatchNorm', nn.BatchNorm2d(out_planes))

        # Relu Floor
        if userelu:
            self.layers.add_module('ReLU', nn.ReLU(inplace=True))

        # Maximum pooling layer, size 2*2
        self.layers.add_module(
            'MaxPool', nn.MaxPool2d(kernel_size=2, stride=2, padding=0))

    # Forward propagation
    def forward(self, x):
        out = self.layers(x)
    # Get output
        return out




In [26]:
# Convolutional network
class ConvNet(nn.Module):
    # Initialize the instance object
    def __init__(self, in_planes, out_planes, num_stages,userelu=False):

        # Initialize the properties inherited from the parent class
        super(ConvNet, self).__init__()

        # Get the size of the input channel
        self.in_planes  = in_planes

        # Get the size of the output channel
        self.out_planes = out_planes

        # Get the number of network layers
        self.num_stages = num_stages

        # Get the number of output channels per layer
        if type(self.out_planes) == int:
            self.out_planes = [self.out_planes for i in range(self.num_stages)]
        assert(type(self.out_planes)==list and len(self.out_planes)==self.num_stages)

        num_planes = [self.in_planes,] + self.out_planes



        conv_blocks = []
        # Define 4-layer convolutional network
        for i in range(self.num_stages):
            if i == (self.num_stages-1):
                conv_blocks.append(
                    ConvBlock(num_planes[i], num_planes[i+1], userelu=userelu))
            else:
                conv_blocks.append(
                    ConvBlock(num_planes[i], num_planes[i+1]))
        # Building a convolutional network
        self.conv_blocks = nn.Sequential(*conv_blocks)

        # Initialize convolutional network parameters
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        # Forward propagation
        out = self.conv_blocks(x)
        # Convert output to 1 dimension
        out = out.view(out.size(0),-1)
        # Get output
        return out


In [27]:
# Build model
# def create_model(opt):
#     return ConvNet(opt)
def Conv64():
    return  ConvNet(in_planes=3,out_planes=[64,64,64,64],num_stages=4,userelu=False)

def Conv128():
    return  ConvNet(in_planes=3,out_planes=[64,64,128,128],num_stages=4,userelu=False)

# Classifiers

In [28]:
class LinearDiag(nn.Module):

    def __init__(self, num_features, bias=False):

        super(LinearDiag, self).__init__()

        weight = torch.FloatTensor(num_features).fill_(1) # initialize to the identity transform
        self.weight = nn.Parameter(weight, requires_grad=True)

        if bias:
            bias = torch.FloatTensor(num_features).fill_(0)
            self.bias = nn.Parameter(bias, requires_grad=True)
        else:
            self.register_parameter('bias', None)

    def forward(self, X):
        assert(X.dim()==2 and X.size(1)==self.weight.size(0))
        out = X * self.weight.expand_as(X)
        if self.bias is not None:
            out = out + self.bias.expand_as(out)
        return out

In [29]:
class FeatExemplarAvgBlock(nn.Module):

    def __init__(self, nFeat):
        super(FeatExemplarAvgBlock, self).__init__()

    def forward(self, features_train, labels_train):
        labels_train_transposed = labels_train.transpose(1,2)

        weight_novel = torch.bmm(labels_train_transposed, features_train)
        weight_novel = weight_novel.div(
            labels_train_transposed.sum(dim=2, keepdim=True).expand_as(weight_novel))

        return weight_novel

In [30]:
class KTN_Classifier(nn.Module):

    def __init__(self,nKall=64, nFeat=64 * 5 * 5, scale_cls=10.0):
        super(KTN_Classifier, self).__init__()

        self.favgblock = FeatExemplarAvgBlock(nFeat)

        weight_base = torch.FloatTensor(nKall, nFeat).normal_(0.0, np.sqrt(2.0 / nFeat))
        self.weight_base = nn.Parameter(weight_base, requires_grad=True)

        self.bias = nn.Parameter(torch.FloatTensor(1).fill_(0), requires_grad=True)

        self.scale_cls = nn.Parameter(torch.FloatTensor(1).fill_(scale_cls), requires_grad=True)


    def get_classification_weights(
            self, Kbase_ids, Knovel_ids, pred, features_support, labels_support):

        batch_size, nKbase = Kbase_ids.size()

        weight_base = self.weight_base[Kbase_ids.view(-1)]
        weight_base = weight_base.view(batch_size, nKbase, -1)

        if features_support is None or labels_support is None:

            return weight_base

        else:

            _, num_train_examples, num_channels = features_support.size()

            nKnovel = labels_support.size(2)

            features_support = F.normalize(
                    features_support, p=2, dim=features_support.dim() - 1, eps=1e-12)

            weight_novel = self.favgblock(features_support, labels_support)
            weight_novel = weight_novel.view(batch_size, nKnovel, num_channels)

            if pred is not None:

                fcw = torch.stack(
                    (pred['pred'][Knovel_ids[0][0]], pred['pred'][Knovel_ids[0][1]], pred['pred'][Knovel_ids[0][2]],
                     pred['pred'][Knovel_ids[0][3]], pred['pred'][Knovel_ids[0][4]]), 0)
                fcw = fcw.view(batch_size, nKnovel, num_channels)

                coefficient = float(5 / labels_support.size()[1])
                weight_novel = weight_novel + fcw * coefficient

            weight_both = torch.cat([weight_base, weight_novel], dim=1)

            return weight_both

    def apply_classification_weights(self, features, cls_weights):

        features = F.normalize(
                features, p=2, dim=features.dim() - 1, eps=1e-12)
        cls_weights = F.normalize(
                cls_weights, p=2, dim=cls_weights.dim() - 1, eps=1e-12)

        cls_scores = self.scale_cls * torch.baddbmm(1.0,self.bias.view(1, 1, 1), 1.0, features,
        cls_weights.transpose(1, 2))
        return cls_scores

    def forward(self, features_query=None, Kbase_ids=None, Knovel_ids=None, pred_fcw = None, features_support=None, labels_support=None):

        cls_weights = self.get_classification_weights(
            Kbase_ids, Knovel_ids, pred_fcw, features_support, labels_support)

        cls_scores = self.apply_classification_weights(
            features_query, cls_weights)

        return cls_scores

# Train

In [31]:
!pip install tqdm

In [32]:
parser = argparse.ArgumentParser()

In [33]:
#********************* Model/Dataset/Path Config *********************#
parser.add_argument('--num_epoch', type=int, default=2,
                    help='number of training epochs')
parser.add_argument('--save_path', default='experiments')
parser.add_argument('--network', type=str, default='Conv128',
                    help='choose which embedding network (Conv64/128) to use')
parser.add_argument('--dataset', type=str, default='miniImageNet',
                    help='choose which classification head to use. miniImageNet, tieredImageNet')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='miniImageNet', type=<class 'str'>, choices=None, help='choose which classification head to use. miniImageNet, tieredImageNet', metavar=None)

In [34]:
#********************* Training Config *********************#
parser.add_argument('--train_nKnovel', type=int, default=0, help='number of novel categories during 1-stage_base training phase')
parser.add_argument('--train_nKbase', type=int, default=64, help='number of base categories during 1-stage_base training phase')
parser.add_argument('--train_nExemplars', type=int, default=0, help='number of support examples per novel category')
parser.add_argument('--train_nTestNovel', type=int, default=0, help='number of query examples for all the novel category')
parser.add_argument('--train_nTestBase', type=int, default=32, help='number of test examples for all the base category')
parser.add_argument('--train_batch_size', type=int, default=8, help='number of episodes per batch')
parser.add_argument('--train_epoch_size', type=int, default=8*1000, help='number of episodes per epoch')

_StoreAction(option_strings=['--train_epoch_size'], dest='train_epoch_size', nargs=None, const=None, default=8000, type=<class 'int'>, choices=None, help='number of episodes per epoch', metavar=None)

In [35]:
#********************* Valing Config *********************#
parser.add_argument('--val_nKnovel', type=int, default=5, help='number of novel categories during 1-stage_base valing phase')
parser.add_argument('--val_nKbase', type=int, default=64, help='number of base categories during 1-stage_base valing phase')
parser.add_argument('--val_nExemplars', type=int, default=1, help='number of support examples per novel category')
parser.add_argument('--val_nTestNovel', type=int, default=15*5, help='number of query examples for all the novel category')
parser.add_argument('--val_nTestBase', type=int, default=15*5, help='number of test examples for all the base category')
parser.add_argument('--val_batch_size', type=int, default=1, help='number of episodes per batch')
parser.add_argument('--val_epoch_size', type=int, default=2000, help='number of batchs per epoch')

_StoreAction(option_strings=['--val_epoch_size'], dest='val_epoch_size', nargs=None, const=None, default=2000, type=<class 'int'>, choices=None, help='number of batchs per epoch', metavar=None)

In [36]:
params = parser.parse_args("")
# all_defaults = {}
# for key in vars(params):
#     all_defaults[key] = parser.get_default(key)
# all_defaults

In [37]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print('using gpu:', '0')

using gpu: 0


In [38]:
save_path1 = os.path.join(params.save_path)
mkdir((save_path1))
save_path2 = os.path.join(params.save_path, params.dataset)
mkdir((save_path2))
params.network = ['Conv64', 'Conv128']
dir_1 = os.path.join(params.save_path, params.dataset,params.network[0])
dir_2 = os.path.join(params.save_path, params.dataset,params.network[1])
mkdir((dir_1))
mkdir((dir_2))

In [39]:
log_file_path_1 = os.path.join(dir_1, "train_log.txt")
log(log_file_path_1, str(vars(params)))
log_file_path_2 = os.path.join(dir_2, "train_log.txt")
log(log_file_path_2, str(vars(params)))

{'num_epoch': 2, 'save_path': 'experiments', 'network': ['Conv64', 'Conv128'], 'dataset': 'miniImageNet', 'train_nKnovel': 0, 'train_nKbase': 64, 'train_nExemplars': 0, 'train_nTestNovel': 0, 'train_nTestBase': 32, 'train_batch_size': 8, 'train_epoch_size': 8000, 'val_nKnovel': 5, 'val_nKbase': 64, 'val_nExemplars': 1, 'val_nTestNovel': 75, 'val_nTestBase': 75, 'val_batch_size': 1, 'val_epoch_size': 2000}
{'num_epoch': 2, 'save_path': 'experiments', 'network': ['Conv64', 'Conv128'], 'dataset': 'miniImageNet', 'train_nKnovel': 0, 'train_nKbase': 64, 'train_nExemplars': 0, 'train_nTestNovel': 0, 'train_nTestBase': 32, 'train_batch_size': 8, 'train_epoch_size': 8000, 'val_nKnovel': 5, 'val_nKbase': 64, 'val_nExemplars': 1, 'val_nTestNovel': 75, 'val_nTestBase': 75, 'val_batch_size': 1, 'val_epoch_size': 2000}


In [40]:
dataset_train = MiniImageNet(phase="train")

Loading mini ImageNet dataset - phase train


In [41]:
dataset_val = MiniImageNet(phase='val')

Loading mini ImageNet dataset - phase val


In [42]:
dataloder_train = FewShotDataloader(
    dataset=dataset_train,
    nKnovel=params.train_nKnovel,
    nKbase=params.train_nKbase,
    nExemplars=params.train_nExemplars,
    nTestNovel=params.train_nTestNovel,
    nTestBase=params.train_nTestBase,
    batch_size=params.train_batch_size,
    num_workers= 4,
    epoch_size=params.train_epoch_size,
)

In [43]:
dataloder_val = FewShotDataloader(
    dataset=dataset_val,
    nKnovel=params.val_nKnovel,
    nKbase=params.val_nKbase,
    nExemplars=params.val_nExemplars,
    nTestNovel=params.val_nTestNovel,
    nTestBase=params.val_nTestBase,
    batch_size=params.val_batch_size,
    num_workers= 4,
    epoch_size=params.val_epoch_size,
)

In [58]:
def modd(par):
    if par == 'Conv64':
        embedding_model = Conv64().cuda()
        classifier = KTN_Classifier(nKall=64, nFeat=64 * 5 * 5, scale_cls=10).cuda()
    elif par == 'Conv128':
        embedding_model = Conv128().cuda()
        classifier = KTN_Classifier(nKall=64, nFeat=128 * 5 * 5, scale_cls=10).cuda()
    else:  # the other backbones are coming soon!
        raise ValueError('Unknown model')

    return embedding_model,classifier
    
    

In [46]:
dir = [dir_1, dir_2]
#save_path = [save_path_1, save_path_2]
parr = ['Conv64', 'Conv128']
log_file_path = [log_file_path_1, log_file_path_2]
best_model = []
for y in range(2):
    embedding_model, classifier = modd(parr[y])
    
    optimizer = torch.optim.SGD([{'params': embedding_model.parameters()},{'params': classifier.parameters()}],
                            lr=0.1, momentum=0.9, weight_decay=5e-4, nesterov=True)
    lambda_epoch = lambda e: 1.0 if e < 20 else (0.06 if e < 40 else 0.012 if e < 50 else (0.0024))
    lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch, last_epoch=-1)

    max_val_acc = 0.0
    max_val_epoch = 0.0

    timer = Timer()
    Loss = torch.nn.CrossEntropyLoss()
    #for l in range(2):
    print("Start training for Model-",params.network[y] )

    for epoch in range(1, params.num_epoch + 1):

        epoch_learning_rate = 0.1
        for param_group in optimizer.param_groups:
            epoch_learning_rate = param_group['lr']

        log(log_file_path[y], 'Train Epoch: {}\tLearning Rate: {:.4f}'.format(epoch, epoch_learning_rate))

        _, _ = [z.train() for z in (embedding_model, classifier)]
        train_accuracies = []
        train_losses = []

        for i, batch in enumerate(tqdm(dataloder_train(epoch)), 1):

            data_train, labels_train = batch[0].cuda(), batch[1].view(params.train_batch_size*params.train_nTestBase).cuda()
            all_Kids, nKbase = batch[2], batch[3].squeeze()[0]

            emb_data_train = embedding_model(data_train.reshape([-1] + list(data_train.shape[-3:])))
            emb_data_train = emb_data_train.reshape(params.train_batch_size,params.train_nTestBase,-1)

            Kbase_ids =  Variable(all_Kids[:, :nKbase].contiguous(), requires_grad=False)

            cls_scores = classifier(features_query=emb_data_train, Kbase_ids=Kbase_ids).view(params.train_batch_size*params.train_nTestBase,-1)

            loss = Loss(cls_scores,labels_train)
            acc= top1accuracy(cls_scores.data, labels_train.data)

            train_accuracies.append(acc.item())
            train_losses.append(loss.item())

            if (i % 200 == 0):
                train_acc_avg = np.mean(np.array(train_accuracies))
                log(log_file_path[y], 'Train Epoch: {}\tBatch: [{}/{}]\tLoss: {:.4f}\tAccuracyBase: {:.2f} % ({:.2f} %)'.format(
                    epoch, i, len(dataloder_train), loss.item(), train_acc_avg, acc))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        lr_scheduler.step()

        _, _ = [z.eval() for z in (embedding_model, classifier)]

        val_accuracies_both = []
        val_accuracies_base = []
        val_accuracies_novel = []

        for i, batch in enumerate(tqdm(dataloder_val(epoch)), 1):

            data_support, labels_support, data_query, labels_query, all_Kids, nKbase = [x.cuda() for x in batch]

            labels_support_one_hot = one_hot(labels_support.reshape(-1)-64,5).unsqueeze(dim=0)

            Kbase_ids = Variable(all_Kids[:, :nKbase].contiguous(), requires_grad=False)

            emb_data_support = embedding_model(data_support.view([-1] + list(data_support.shape[-3:])))
            emb_data_support = emb_data_support.view(params.val_batch_size,params.val_nKnovel,-1)

            emb_data_query = embedding_model(data_query.view([-1] + list(data_query.shape[-3:])))
            emb_data_query = emb_data_query.view(params.val_batch_size, params.val_nTestBase + params.val_nTestNovel, -1)

            cls_scores = classifier(features_query=emb_data_query, Kbase_ids=Kbase_ids,
                                    features_support=emb_data_support, labels_support=labels_support_one_hot).view(
                                        params.val_batch_size*(params.val_nTestBase+params.val_nTestNovel),-1)

            accuracyBoth, accuracyBase, accuracyNovel = top1accuracy_all(cls_scores, labels_query.view(-1), nKbase)

            val_accuracies_both.append(accuracyBoth.item())
            val_accuracies_base.append(accuracyBase.item())
            val_accuracies_novel.append(accuracyNovel.item())

        val_acc_both = np.mean(np.array(val_accuracies_both))
        val_acc_both_ci95 = 1.96 * np.std(np.array(val_accuracies_both)) / np.sqrt(params.val_epoch_size)

        val_acc_base = np.mean(np.array(val_accuracies_base))
        val_acc_base_ci95 = 1.96 * np.std(np.array(val_accuracies_base)) / np.sqrt(params.val_epoch_size)

        val_acc_novel = np.mean(np.array(val_accuracies_novel))
        val_acc_novel_ci95 = 1.96 * np.std(np.array(val_accuracies_novel)) / np.sqrt(params.val_epoch_size)

        if val_acc_novel > max_val_acc:
            max_val_acc = val_acc_novel
            max_val_epoch = epoch
            torch.save({'embedding': embedding_model.state_dict(), 'classifier': classifier.state_dict()}, os.path.join(params.save_path, params.dataset,params.network[y], '1_stage_best_model.pth'))
            log(log_file_path[y], 'Best model saving!!!')
            log(log_file_path[y],
                'Val_Epoch: [{}/{}]\tAccuracyBoth: {:.2f} +- {:.2f} %\tAccuracyBase: {:.2f} +- {:.2f} %\tAccuracyNovel: {:.2f} +- {:.2f} %'.format(
                epoch, params.num_epoch,
                val_acc_both, val_acc_both_ci95,
                val_acc_base, val_acc_base_ci95,
                val_acc_novel, val_acc_novel_ci95))
        else:
            log(log_file_path[y],
                'Val_Epoch: [{}/{}]\tAccuracyBoth: {:.2f} +- {:.2f} %\tAccuracyBase: {:.2f} +- {:.2f} %\tAccuracyNovel: {:.2f} +- {:.2f} %'.format(
                    epoch, params.num_epoch,
                    val_acc_both, val_acc_both_ci95,
                    val_acc_base, val_acc_base_ci95,
                    val_acc_novel, val_acc_novel_ci95))

        torch.save({'embedding': embedding_model.state_dict(), 'classifier': classifier.state_dict()},
                os.path.join(params.save_path, params.dataset,params.network[y], '1_stage_last_model.pth'))

        log(log_file_path[y], 'Elapsed Time: {}/{}\n'.format(timer.measure(), timer.measure(epoch / float(params.num_epoch))))

    print("END training for Model-",params.network[y] )
    log(log_file_path[y],
            'Best model saving!!!\tBest_Epoch: [{}/{}]\tAccuracyNovel: {:.2f} %'.format(
            max_val_epoch, params.num_epoch,max_val_acc))


  0%|          | 0/1000 [00:00<?, ?it/s]

Start training for Model- Conv64
Train Epoch: 1	Learning Rate: 0.1000


 20%|██        | 201/1000 [00:30<01:59,  6.71it/s]

Train Epoch: 1	Batch: [200/1000]	Loss: 2.5729	AccuracyBase: 22.33 % (32.03 %)


 40%|████      | 401/1000 [00:59<01:29,  6.66it/s]

...

Train Epoch: 2	Batch: [800/1000]	Loss: 1.3764	AccuracyBase: 56.33 % (61.72 %)


100%|██████████| 1000/1000 [02:22<00:00,  8.29it/s]

Train Epoch: 2	Batch: [1000/1000]	Loss: 1.3786	AccuracyBase: 57.04 % (62.50 %)


100%|██████████| 2000/2000 [02:37<00:00, 12.66it/s]

Best model saving!!!
Val_Epoch: [2/2]	AccuracyBoth: 34.51 +- 0.20 %	AccuracyBase: 51.57 +- 0.25 %	AccuracyNovel: 46.46 +- 0.42 %
Elapsed Time: 10m/10m

END training for Model- Conv128
Best model saving!!!	Best_Epoch: [2/2]	AccuracyNovel: 46.46 %


#### Test Only Vision

In [47]:
parser = argparse.ArgumentParser()

In [48]:
#********************* Model/Dataset/Path Config *********************#
parser.add_argument('--save_path', default='experiments')
parser.add_argument('--network', type=str, default='Conv64',
                    help='choose which embedding network to use')
parser.add_argument('--dataset', type=str, default='miniImageNet',
                    help='choose which classification head to use. miniImageNet, tieredImageNet')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='miniImageNet', type=<class 'str'>, choices=None, help='choose which classification head to use. miniImageNet, tieredImageNet', metavar=None)

In [49]:
#********************* Testing Config *********************#
parser.add_argument('--test_nKnovel', type=int, default=5, help='number of novel categories during 1-stage_base testing phase')
parser.add_argument('--test_nKbase', type=int, default=64, help='number of base categories during 1-stage_base testing phase')
parser.add_argument('--test_nExemplars', type=int, default=5, help='number of support examples per novel category')
parser.add_argument('--test_nTestNovel', type=int, default=15*5, help='number of query examples for all the novel category')
parser.add_argument('--test_nTestBase', type=int, default=15*5, help='number of test examples for all the base category')
parser.add_argument('--test_batch_size', type=int, default=1, help='number of episodes per batch')
parser.add_argument('--test_epoch_size', type=int, default=600, help='number of batchs per epoch')

_StoreAction(option_strings=['--test_epoch_size'], dest='test_epoch_size', nargs=None, const=None, default=600, type=<class 'int'>, choices=None, help='number of batchs per epoch', metavar=None)

In [50]:
params = parser.parse_args("")
all_defaults = {}
for key in vars(params):
    all_defaults[key] = parser.get_default(key)
all_defaults

{'dataset': 'miniImageNet',
 'network': 'Conv64',
 'save_path': 'experiments',
 'test_batch_size': 1,
 'test_epoch_size': 600,
 'test_nExemplars': 5,
 'test_nKbase': 64,
 'test_nKnovel': 5,
 'test_nTestBase': 75,
 'test_nTestNovel': 75}

In [51]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print('using gpu:', '0')

using gpu: 0


In [52]:
#save_path = os.path.join(params.save_path, params.dataset, params.network)
#mkdir((save_path))

In [53]:
dataset_test = MiniImageNet(phase='test')
dataloder_test = FewShotDataloader(
    dataset=dataset_test,
    nKnovel=params.test_nKnovel,
    nKbase=params.test_nKbase,
    nExemplars=params.test_nExemplars,
    nTestNovel=params.test_nTestNovel,
    nTestBase=params.test_nTestBase,
    batch_size=params.test_batch_size,
    num_workers= 4,
    epoch_size=params.test_epoch_size,
)

Loading mini ImageNet dataset - phase test


In [54]:
def modd(par):
    
    for par in range(2):
        if params.network[par] == 'Conv64':
            embedding_model = Conv64().cuda()
            classifier = KTN_Classifier(nKall=64, nFeat=64 * 5 * 5, scale_cls=10).cuda()
        elif params.network[par] == 'Conv128':
            embedding_model = Conv128().cuda()
            classifier = KTN_Classifier(nKall=64, nFeat=128 * 5 * 5, scale_cls=10).cuda()
        else:  # the other backbones are coming soon!
            raise ValueError('Unknown model')

        return embedding_model,classifier 
    
        

In [60]:
best_model = []
for x in range(2):
    embedding_model, classifier = modd(parr[x])
    best = torch.load(os.path.join(dir[x],'1_stage_best_model.pth'))
    best_model.append(best)
    embedding_model.load_state_dict(best_model[x]['embedding'])
    classifier.load_state_dict(best_model[x]['classifier'])
    embedding_model.load_state_dict(best_model[x]['embedding'])
    classifier.load_state_dict(best_model[x]['classifier'])
    _, _ = [z.eval() for z in (embedding_model, classifier)]
    test_accuracies_both = []
    test_accuracies_base = []
    test_accuracies_novel = []

    for i, batch in enumerate(tqdm(dataloder_test()), 1):

        data_support, labels_support, data_query, labels_query, all_Kids, nKbase = [x.cuda() for x in batch]

        labels_support_one_hot = one_hot(labels_support.reshape(-1) - 64, 5).unsqueeze(dim=0)

        Kbase_ids = Variable(all_Kids[:, :nKbase].contiguous(), requires_grad=False)
        Knovel_ids = Variable(all_Kids[:, nKbase:].contiguous(), requires_grad=False)

        emb_data_support = embedding_model(data_support.view([-1] + list(data_support.shape[-3:])))
        emb_data_support = emb_data_support.view(params.test_batch_size, params.test_nExemplars*params.test_nKnovel, -1)

        emb_data_query = embedding_model(data_query.view([-1] + list(data_query.shape[-3:])))
        emb_data_query = emb_data_query.view(params.test_batch_size, params.test_nTestBase + params.test_nTestNovel,-1)

        cls_scores = classifier(features_query=emb_data_query, Kbase_ids=Kbase_ids,Knovel_ids = Knovel_ids,
                            features_support=emb_data_support, labels_support=labels_support_one_hot).view(
            params.test_batch_size * (params.test_nTestBase + params.test_nTestNovel), -1)

        accuracyBoth, accuracyBase, accuracyNovel = top1accuracy_all(cls_scores, labels_query.view(-1), nKbase)

        test_accuracies_both.append(accuracyBoth.item())
        test_accuracies_base.append(accuracyBase.item())
        test_accuracies_novel.append(accuracyNovel.item())

100%|██████████| 600/600 [00:50<00:00, 11.91it/s]


In [61]:
test_acc_both = np.mean(np.array(test_accuracies_both))
test_acc_both_ci95 = 1.96 * np.std(np.array(test_accuracies_both)) / np.sqrt(params.test_epoch_size)

test_acc_base = np.mean(np.array(test_accuracies_base))
test_acc_base_ci95 = 1.96 * np.std(np.array(test_accuracies_base)) / np.sqrt(params.test_epoch_size)

test_acc_novel = np.mean(np.array(test_accuracies_novel))
test_acc_novel_ci95 = 1.96 * np.std(np.array(test_accuracies_novel)) / np.sqrt(params.test_epoch_size)

In [62]:
print('AccuracyBoth: {:.2f} +- {:.2f} %\tAccuracyBase: {:.2f} +- {:.2f} %\tAccuracyNovel: {:.2f} +- {:.2f} %'.format(
            test_acc_both, test_acc_both_ci95, test_acc_base, test_acc_base_ci95,test_acc_novel, test_acc_novel_ci95))

AccuracyBoth: 45.27 +- 0.40 %	AccuracyBase: 51.19 +- 0.47 %	AccuracyNovel: 65.29 +- 0.69 %


#### Testing Visiual Knowledge

In [63]:
parser = argparse.ArgumentParser()

parser.add_argument('--num_epoch', type=int, default=1,
                    help='number of training epochs')
parser.add_argument('--save_path', default='experiments')
parser.add_argument('--network', type=str, default='Conv64',
                    help='choose which embedding network to use')
parser.add_argument('--dataset', type=str, default='miniImageNet',
                    help='choose which classification head to use. miniImageNet, tieredImageNet')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='miniImageNet', type=<class 'str'>, choices=None, help='choose which classification head to use. miniImageNet, tieredImageNet', metavar=None)

In [64]:
#***********************************************************************************************
parser.add_argument('--test_nKnovel', type=int, default=5, help='number of novel categories during 1-stage_base testing phase')
parser.add_argument('--test_nKbase', type=int, default=64, help='number of base categories during 1-stage_base testing phase')
parser.add_argument('--test_nExemplars', type=int, default=1, help='number of support examples per novel category')
parser.add_argument('--test_nTestNovel', type=int, default=15*5, help='number of query examples for all the novel category')
parser.add_argument('--test_nTestBase', type=int, default=15*5, help='number of test examples for all the base category')
parser.add_argument('--test_batch_size', type=int, default=1, help='number of episodes per batch')
parser.add_argument('--test_epoch_size', type=int, default=600, help='number of batchs per epoch')

_StoreAction(option_strings=['--test_epoch_size'], dest='test_epoch_size', nargs=None, const=None, default=600, type=<class 'int'>, choices=None, help='number of batchs per epoch', metavar=None)

In [65]:
params = parser.parse_args("")
all_defaults = {}
for key in vars(params):
    all_defaults[key] = parser.get_default(key)
all_defaults

{'dataset': 'miniImageNet',
 'network': 'Conv64',
 'num_epoch': 1,
 'save_path': 'experiments',
 'test_batch_size': 1,
 'test_epoch_size': 600,
 'test_nExemplars': 1,
 'test_nKbase': 64,
 'test_nKnovel': 5,
 'test_nTestBase': 75,
 'test_nTestNovel': 75}

In [66]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print('using gpu:', '0')

using gpu: 0


In [67]:
#save_path = os.path.join(params.save_path, params.dataset, params.network)
model = "Models/Conv64_fc.pred"
# file = open(model, "r")

In [68]:
# Only for Conv64 Model
pred_fcw = torch.load(model)



In [69]:
dataset_test = MiniImageNet(phase='test')
dataloder_test = FewShotDataloader(
    dataset=dataset_test,
    nKnovel=params.test_nKnovel,
    nKbase=params.test_nKbase,
    nExemplars=params.test_nExemplars,
    nTestNovel=params.test_nTestNovel,
    nTestBase=params.test_nTestBase,
    batch_size=params.test_batch_size,
    num_workers= 4,
    epoch_size=params.test_epoch_size,
)

Loading mini ImageNet dataset - phase test


In [71]:
def mod(par):
    if par == 'Conv64':
        embedding_model = Conv64().cuda()
        classifier = KTN_Classifier(nKall=64, nFeat=64 * 5 * 5, scale_cls=10).cuda()
    else:  # the other backbones are coming soon!
        raise ValueError('Unknown models')
    return embedding_model,classifier   
  

In [72]:

#torch.load(os.path.join(dir[par],'1_stage_best_model.pth'))
embedding_model,classifier = mod('Conv64')

In [73]:
best_model = torch.load(os.path.join(dir_1,'1_stage_best_model.pth'))

embedding_model.load_state_dict(best_model['embedding'])
classifier.load_state_dict(best_model['classifier'])

<All keys matched successfully>

In [74]:
_, _ = [x.eval() for x in (embedding_model, classifier)]

In [75]:
test_accuracies_both = []
test_accuracies_base = []
test_accuracies_novel = []

In [77]:
for i, batch in enumerate(tqdm(dataloder_test()), 1):

    data_support, labels_support, data_query, labels_query, all_Kids, nKbase = [x.cuda() for x in batch]
    
    labels_support_one_hot = one_hot(labels_support.reshape(-1) - 64, 5).unsqueeze(dim=0)
    
    Kbase_ids = Variable(all_Kids[:, :nKbase].contiguous(), requires_grad=False)
    Knovel_ids = Variable(all_Kids[:, nKbase:].contiguous(), requires_grad=False)
    
    emb_data_support = embedding_model(data_support.view([-1] + list(data_support.shape[-3:])))
    emb_data_support = emb_data_support.view(params.test_batch_size, params.test_nExemplars*params.test_nKnovel, -1)
    #emb_data_supportt = emb_data_support.view(8000,8000,1)
    emb_data_query = embedding_model(data_query.view([-1] + list(data_query.shape[-3:])))
    emb_data_query = emb_data_query.view(params.test_batch_size, params.test_nTestBase + params.test_nTestNovel,-1)
    #emb_data_queryy = emb_data_query.view(8000, 8000, 1)

    cls_scores = classifier(features_query=emb_data_query, Kbase_ids=Kbase_ids,Knovel_ids = Knovel_ids,pred_fcw = pred_fcw,
                            features_support=emb_data_support,
                            labels_support=labels_support_one_hot).view(params.test_batch_size * (params.test_nTestBase + params.test_nTestNovel), -1)
    
    accuracyBoth, accuracyBase, accuracyNovel = top1accuracy_all(cls_scores, labels_query.view(-1), nKbase)

    test_accuracies_both.append(accuracyBoth.item())
    test_accuracies_base.append(accuracyBase.item())
    test_accuracies_novel.append(accuracyNovel.item())

100%|██████████| 600/600 [00:44<00:00, 13.63it/s]


In [78]:
test_acc_both = np.mean(np.array(test_accuracies_both))
test_acc_both_ci95 = 1.96 * np.std(np.array(test_accuracies_both)) / np.sqrt(params.test_epoch_size)

In [79]:
test_acc_base = np.mean(np.array(test_accuracies_base))
test_acc_base_ci95 = 1.96 * np.std(np.array(test_accuracies_base)) / np.sqrt(params.test_epoch_size)

In [80]:
test_acc_novel = np.mean(np.array(test_accuracies_novel))
test_acc_novel_ci95 = 1.96 * np.std(np.array(test_accuracies_novel)) / np.sqrt(params.test_epoch_size)

In [81]:
print('AccuracyBoth: {:.2f} +- {:.2f} %\tAccuracyBase: {:.2f} +- {:.2f} %\tAccuracyNovel: {:.2f} +- {:.2f} %'.format(
            test_acc_both, test_acc_both_ci95, test_acc_base, test_acc_base_ci95,test_acc_novel, test_acc_novel_ci95))

AccuracyBoth: 30.38 +- 0.32 %	AccuracyBase: 49.88 +- 0.44 %	AccuracyNovel: 44.52 +- 0.75 %
